In [1]:
import os
import numpy as np
#from Starter.Detection import Detector, Network
from Stochastic import Princess as SP
from Starter.AstroModel import AstroModel
from Starter import Detection
from Individual.Individual_analysis import IndividualAnalysis as IA



<div>
<img src="./Wiki/Princess_logo.png" width="300"/>
</div>

# Princess: Guide for a first calculation


This file goes step-by-step through the calculation of an astrophysical background starting from a CBC catalogue.
For mode details about the code structure and basic calculations please visit the README.md file.
Princess as two companioin paper which give more details on the physica behind these calculations.

This toolkit aims to be user friendly and useful to the collaboration. If you have any comments, issues or requests please contact the administrator (_caroleperigois@outlook.com_).

The calculation of the background goes through four main step, defining the four sections of this file.
1. Prepare your model
2. Prepare your detectors and Networks
3. Calculate the background
4. Analyse our background

## 1. Prepare your model: 
All population synthesis codes may have different outputs. In the next steps the code will re-build catalogues insuring that it contains all the required parameters for the next steps of calculation.
Your astrophysical model will be set in a class Princess.Astromodel and takes in entry several parameter.
* `cat_name`: (_str_) is the name you want to use for your model
* `original_cat_path`: (_str_) path to your original astrophysical catalogue
* `cat_sep`: (_str_) separator used in your original catalogue (default is tab)
* `index_column`: (_bool_) does your original file contain a columns with indexes (default is None)
* `flags`: (_dict_) this option allow to differenciate different types of CBCs is a Model. If you add this option you need to set up a dictionnary of the different categories. For example in the checks the original catalogue contain a column called 'flag' wher can be found identifiers 1 for isolated BBH and 2 for cluster BBH. Therefore the dictionnary looks like Flags = {'1': 'Iso', '2':'Cluster'}. In the next steps the code will build two catalogues out from the initial model (default is None).



In [2]:
path_2_catalogue_folder =  '/home/perigois/Documents/GreatWizardOz/Bkg_uncertainties/'
Flags = {'1': 'Orig', '2':'Exch'}
Cat = 'Test_week.dat'
Astromodel = AstroModel(name= 'Princess_Test', original_path = path_2_catalogue_folder+Cat, 
                         sep = " ", index_column = None, spin_option = 'rand_Dynamics')

0


If your original catalogue do not have header you can set one using the method makeHeader on your model. In the next liste are the labels allowed by the code, please note that for the masse you will need to have or the chirp mass and the mass ratio, or the two masses m1 and m2.
* Mc : Chirp mass in the source frame [Msun]
* q : mass ratio in the source frame
* m1 : mass of the first component (m1>m2) in the source frame [Msun]
* m2 : mass of the secondary component (m2<m1) in the source frame [Msun]
* Xeff : effective spin
* s1 : individual spin factor of the first component
* s2 : individual spin factor of the second component
* theta1 : angle between the first spin component and the angular momentum of the binary [rad]
* theta2 : angle between the second spin component and the angular momentum of the binary [rad] 
* a0 : semi-major axis at the formaiton of the second compact object [Rsun]
* e0 : eccentricity at the formation of the second compact object  
* inc : inclinaison angle [rad]       
* zm : redshift of merger
* zf : redshift of formation  
* Dl : luminosity distance [Mpc]
* flag : this columns cam contain a flag to differenciate sources

Set spin option:
    True if you want to include spin in your calculations of the waveforms and background and you have the spin in your catalogue
    'Zero' if you don't want to use the spin, the code set all spins to 0.
    Model if you want Princess to generate spin values - Option available later -

Finally the MakeCat method generate the catalogue with all requiered parameters

In [3]:
#Astromodel.makeCat()

print('Astromodel loaded and ready :)')

Astromodel loaded and ready :)


## 2. Detectors and Networks
In this part is detailed the context of the study starting by defining the range of frequency `Freq` and the waveforms `WF_approx` to use. In this version the range has to be linear in the future specific function will be added to allow log scale, in particular for LISA band. The available waveforms are the ones define in PyCBC and the analytic one from Ajith2011. The calculation with Ajith waveforms is computationnaly more expensive and therefore not recommended.   

In [4]:
Freq_2G = np.linspace(10, 1000, 991)
Freq_3G = np.linspace(1, 1000, 1000)
WF_approx = "IMRPhenomD"

In Princess two classes has been build for this purpose in the file _Starter/Detection_ in order to define Detectors and combine them to build a Network. The Detector class takes in entry several parameters:
* `name`: (_str_) is the name you give to the detector
* `Pycbc`: (_bool_) True if the sensitivity is available in PyCBC, else False
* `psd_file`: (_str_) name of the sensitivity in PyCBC, or file where your sensitivity is stored
* `freq`: (_np.array_) frequency range of the study

In [5]:
H = Detection.Detector(name = 'H', origin = 'Pycbc', configuration = 'H', psd_file = 'aLIGODesignSensitivityP1200087', freq = Freq_2G)
L = Detection.Detector(name = 'L', origin = 'Pycbc', configuration = 'L', psd_file = 'aLIGODesignSensitivityP1200087', freq = Freq_2G)
V = Detection.Detector(name = 'V', origin = 'Pycbc', configuration = 'V', psd_file = 'AdVDesignSensitivityP1200087', freq = Freq_2G)

In [6]:
ET = Detection.Detector(name = 'ET', configuration = 'ET', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)
CE1 = Detection.Detector(name = 'CE1', origin = 'Princess', configuration = 'H', psd_file = 'CE_20km', freq = Freq_3G)
CE2 = Detection.Detector(name = 'CE2', origin = 'Princess', configuration = 'L', psd_file = 'CE_40km', freq = Freq_3G)

The second class Network allow to combine different detectors to build a Network.A Network takes in entry: 
* `net_name`: (_str_) Name of the network
* `compo`: (_list of detectors_) List of the detectors in the network.
* `pic_file`: (_str_) link to the file of the power integrated curve.
* `efficiency`: (_float_) between 0 and 1 define the effective time of observation of the Network. For example during O3a, in the Hanford-Livinstone-Virgo network only 50% of the data can be used with the three pipelines. The rest of the time at least on detector pipeline was unusuable.
* `SNR_thrs`: (_int_ or _float_) Define the SNR threshold for which we assume a source is detectable
* `SNR_sub`: (_int_ or _float_) Define the SNR threshold to substract the sources. For example its commonly assumed that in HLV all source with an SNR above 8 are resolved. However for a reason of parameter uncertainty the calculation of the residual background is done by subtracting only source with a SNR above 12. 


**If only one detector is used in the study it still has to be set as a detector.**

The variable `Networks` gather all the networks used in the study.


In [7]:
HLV_Des = Detection.Network(name = 'HLV',compo=[H,L,V], pic_file = 'AuxiliaryFiles/PICs/Design_HLV_flow_10.txt' , efficiency = 0.5,SNR_thrs = 12 )
Networks_2G = [HLV_Des]

In [8]:
ETn = Detection.Network(name='ET', compo=[ET], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12)
twoCE = Detection.Network(name='2CE', compo=[CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12)
ET2CE = Detection.Network(name='ET2CE', compo=[ET,CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12)
Networks_3G = [ETn, twoCE, ET2CE]

Finally the method `compute_SNR`, compute the SNR of each sources of the model, and update the catalogue(s) with a new parameter column named by `net_name` containing the SNR in the corresponding network.

In [9]:
#Astromodel.compute_SNR_opt(Networks = Networks_3G, freq = Freq_3G, approx = WF_approx)

## 5. Individual analysis 
This package of the code aims to predict individual observation from LIGO-Virgo from a predefined astrophysical model. As for the background computation the user has to predefine his analysis with the _class_ 'IndividualAnalysis'. This class takes in entry the limits of the study and the assumption on GW events observations.

* `name`: (_str_) Name used to label the output.
* `params` : (_dict_) parameters the user want to compare and the range [minimum, maximum, bins]. Default are : {'m1': [0, 100, 100], 'q': [0, 1, 20], 'zm': [0, 5, 30]}
* `iteration` : (_int_) number of iteration to extract the errors on observations
* `Network` : (Network from _class_ 'Network') network to use in the study
* `binary_type` : (_list_ of _str_) type of binaries to be compared. Default is ['BBH', 'BNS', 'NSBH'].
* `pastro_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 0.
* `SNR_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 0.
* `FAR_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 2.
* `iteration` : (_int_) The number of iteration used to set up the position in the sky, this is used to extract an error on the number of detections.

The method `Full_Analysis` is used to start the computation of the analysis.

In [10]:
#Analysis = IA(name = 'PRINCESS_check', params =  {'m1': [0, 100, 100], 'm2': [0, 100, 100], 'Mc': [0, 70, 100]},
#                             Networks = Networks_3G, binary_type = ['BBH'], iteration = 1)
#Analysis.Full_Analysis(Model = Astromodel, update_file = False)

## 4. Calculate the corresponding background:

Prepare the calculation of the background with the class `Princess` :
* `freq`: (_np.array_) frequency range, preferentially the one used since the beginning. In linear scale before the LISA update
* `approx`: (_str_) waveform approximation
* `freq_ref`: (_list of float_) list of frequency of interest for the study, is usually 10Hz for 3G detectors and 25Hz for 2G.

Then the calculation is done by using the method `Omega_pycbc`

In [11]:
Zelda = SP.Princess(Freq_3G, astromodel = Astromodel, approx = WF_approx, Omega_ana_freq = [25.], Networks = Networks_3G, inclination = "Rand")
Zelda.Make_Ana_Output()
Zelda.Omega_pycbc(Networks= Networks_3G)

{'Princess_Test.dat':              Total   ET  2CE ET2CE
N_source       NaN  NaN  NaN   NaN
Omg_25.0_Hz    NaN  NaN  NaN   NaN
SNR_Total      NaN  NaN  NaN   NaN
SNR_Residual   NaN  NaN  NaN   NaN}
 ***   GW COMPUTATION   ***         0/2271


/home/perigois/PycharmProjects/Princess/Stochastic/Princess.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event['inc'] = np.arccos(r.inc.values)


           ■ ■■ ■             2%    45/2271
          ■■■■■■■■            4%    90/2271
          ■ ■  ■ ■            6%    135/2271
         ■        ■           8%    180/2271
         ■        ■           10%   225/2271
          ■■■■ ■■■            12%   270/2271
         ■    ■  ■■           14%   315/2271
        ■          ■          16%   360/2271
      ■■            ■         18%   405/2271
 ■   ■              ■■        20%   450/2271
  ■■■                 ■       22%   495/2271
 ■                    ■       24%   540/2271
  ■                  ■        26%   585/2271
 ■         ■■     ■■  ■       28%   630/2271
  ■       ■  ■   ■ ■  ■       30%   675/2271
   ■      ■   ■■■ ■■ ■        32%   720/2271
    ■■ ■■ ■  ■   ■ ■■         34%   765/2271
    ■ ■ ■ ■  ■   ■ ■■         36%   810/2271
   ■  ■ ■  ■ ■   ■ ■■■        38%   855/2271
■■■■   ■ ■ ■       ■ ■        40%   900/2271
 ■    ■   ■   ■■  ■ ■         42%   945/2271
 ■     ■ ■ ■     ■■           44%   990/2271
  ■■  ■■■■■ 

## 4. Analyse the background
This part of the code aims to extract reference values for the predicted background. Usual values are the amplitude at 10 and 25 Hz, the SNR, the number of resolved soures, the ratio of detected sources, and the ratio between residuals and total backgrouns at a reference value.

In [12]:
ET_alone = Detection.Detector(name = 'E1', configuration = 'E1E2', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)
E1 = Detection.Detector(name = 'E1', configuration = 'E1', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)
E2 = Detection.Detector(name = 'E2', configuration = 'E2', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)
E3 = Detection.Detector(name = 'E3', configuration = 'E3', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)

CE1 = Detection.Detector(name = 'CE1', origin = 'Princess', configuration = 'CE1', psd_file = 'CE_20km', freq = Freq_3G)
CE2 = Detection.Detector(name = 'CE2', origin = 'Princess', configuration = 'CE2', psd_file = 'CE_40km', freq = Freq_3G)
ETn = Detection.Network(name='ET', compo=[E1,E2,E3], pic_file='AuxiliaryFiles/PICs/ET.txt',SNR_thrs=12, duration = 1.)
twoCE = Detection.Network(name='2CE', compo=[CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12, duration = 1.)
ET2CE = Detection.Network(name='ET2CE', compo=[E1,E2,E3, CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12, duration = 1)
Networks_3G_bkg = [ETn, twoCE, ET2CE]
Zelda.Analysis(Networks = Networks_3G_bkg)
Zelda.Write_results()

2.2242636412261696
28.528867350417446
2.2242636412261696   ET
2.2242636412261696
28.528867350417446
                               Total                     ET  2CE ET2CE
N_source                        2271                    750   56     9
Omg_25.0_Hz   2.2528011750333143e-11  1.804621533459179e-12  NaN   NaN
SNR_Total                        NaN              28.528867  NaN   NaN
SNR_Residual                     NaN               2.224264  NaN   NaN
0.1451496734864558
51.40557620961355
0.1451496734864558   2CE
0.1451496734864558
51.40557620961355
                               Total                     ET  \
N_source                        2271                    750   
Omg_25.0_Hz   2.2528011750333143e-11  1.804621533459179e-12   
SNR_Total                        NaN              28.528867   
SNR_Residual                     NaN               2.224264   

                                2CE ET2CE  
N_source                         56     9  
Omg_25.0_Hz   6.505312283622461e-14   NaN 